In [1]:
import keras
import keras.backend as K

import tensorflow as tf

from keras.layers import Add, Concatenate

Using TensorFlow backend.


In [181]:
def _drop_mod(embeddings, normalize):
    def _mod_drop_train(contextual, residual):
        keep_configs = K.constant([[0, 1], 
                                   [1, 0], 
                                   [1, 1]], dtype='float32')

        # scale by 1.0 / keep_prob
        keep_configs_probs = K.mean(keep_configs, axis=0, keepdims=True)
        keep_configs *= 1.0 / keep_configs_probs

        # [batch_size, sen_length]
        selectors = K.random_uniform(K.shape(contextual)[:2], 0, 3, 'int32')

        # [batch_size, sen_length, 2, 1]
        keep = K.expand_dims(K.gather(keep_configs, selectors))

        # [batch_size, sen_length, 2, 2*emb_dim]
        stacked_embeddings = K.stack([contextual, residual], axis=2)

        # [batch_size, sen_length, 2*emb_dim]
        return keep, selectors, K.sum(keep * stacked_embeddings, axis=2)

    def _mod_drop_test(contextual, residual):
        return K.constant([[1, 1]]), K.constant([[1]], 'int32'), Add()([contextual, residual])

    contextual, residual = embeddings
    # contextual: [batch, sen_length, 2*emb_dim]
    # residual: [batch, sen_length, emb_dim]
    residual = Concatenate()([residual, residual])

    if normalize:
        # [batch_size, sen_length, 1]
        residual_norm = tf.norm(residual, keepdims=True)
        # [batch_size, sen_length, 2*emb_dim]
        unit_contextual = K.l2_normalize(contextual, axis=-1)
        contextual = unit_contextual * residual_norm

    return K.switch(K.learning_phase(),
                    _mod_drop_train(contextual, residual),
                    _mod_drop_test(contextual, residual))

In [182]:
def build_model():
    contextual = K.placeholder(shape=(None, None, 2))
    residual = K.placeholder(shape=(None, None, 1))

    output = _drop_mod([contextual, residual], normalize=False)
    return [contextual, residual], output

In [183]:
TRAINING_PHASE = 1
TEST_PHASE = 0

fixture = {
    'contextual': [[[1, 1]]],
    'residual': [[[2]]],
    'expected': {
        TRAINING_PHASE: [[[3, 3]]],
        TEST_PHASE: [[[3, 3]]],
    } 
}

In [217]:
with tf.Graph().as_default():
    [contextual, residual], output = build_model()
    phase = K.learning_phase()
    
    with tf.Session() as sess:
        K.set_session(sess)
        print(sess.run(output, feed_dict={
            contextual: fixture['contextual'],
            residual: fixture['residual'],
            phase: TRAINING_PHASE
        }))

(array([[[[0. ],
         [1.5]]]], dtype=float32), array([[0]], dtype=int32), array([[[3., 3.]]], dtype=float32))


In [186]:
with tf.Graph().as_default():
    [contextual, residual], joint = build_model()
    phase = K.learning_phase()
    
    with tf.Session() as sess:
        K.set_session(sess)
        K.set_learning_phase(TEST_PHASE)
        real = joint.eval(feed_dict={
            contextual: fixture['contextual'],
            residual: fixture['residual'],
            phase: TEST_PHASE
        })
        print(real)

AttributeError: 'tuple' object has no attribute 'eval'